In [1]:
import os
from google.cloud import speech
import io
import wave
from pydub import AudioSegment
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Speech2Text.json'
client = speech.SpeechClient()

C:\Users\quang\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
bucket_name = 'speech-to-text_stech'
Name = 'Audio_wav'

In [3]:
from google.cloud import storage

# tổng hợp bucket hiện có
def list_buckets():
    """Lists all buckets."""

    storage_client = storage.Client()
    buckets = storage_client.list_buckets()

    for bucket in buckets:
        print(bucket.name)

# # tạo 1 bucket có tên
# def create_bucket_class_location(bucket_name):
#     """
#     Create a new bucket in the US region with the coldline storage
#     class
#     """
#     # bucket_name = "your-new-bucket-name"

#     storage_client = storage.Client()

#     bucket = storage_client.bucket(bucket_name)
#     bucket.storage_class = "COLDLINE"
#     new_bucket = storage_client.create_bucket(bucket, location="us")

#     print(
#         "Created bucket {} in {} with storage class {}".format(
#             new_bucket.name, new_bucket.location, new_bucket.storage_class
#         )
#     )
#     return new_bucket

# Upload large file tới Cloud, nếu tên bị trùng sẽ ghi đè
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [4]:
def frame_rate_channel(audio_file_name):
    print(audio_file_name)
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels
    
def Config_GGC(sample_rate_hertz = 44100,
                audio_channel_count = 1,
                model = None,
                enable_automatic_punctuation=True):
    if model != None:
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=enable_automatic_punctuation,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
            model = model,
        )
    else :
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=True,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced

def Transcribe_Short_Audio(Audio_wav,config_wav_enhanced):
    client = speech.SpeechClient()
    with io.open(Audio_wav, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    # print(type(audio))
    
    response = client.recognize(config=config_wav_enhanced, audio=audio)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        alter=ConvertDate(alternative.transcript+ '\n')
#         print("-" * 20)
#         print("First alternative of result {}".format(i))
#         print("Transcript: {}".format(alternative.transcript))
        text.append(alter)
    return text

def Transcribe_Long_Audio(Audio_wav,config_wav_enhanced,
                        bucket_name = 'speech_to_text_stech',
                        Name = 'Audio_wav'):
    client = speech.SpeechClient()
    Audio_name = Audio_wav.split('/')[-1].split('.')[0]
    
    upload_blob(bucket_name,Audio_wav,Name)
    
    media_uri = "gs://{}/{}".format(bucket_name,Name)
    long_audi_wav = speech.RecognitionAudio(uri=media_uri)
    
    
    operations = client.long_running_recognize(
        config = config_wav_enhanced,
        audio = long_audi_wav
    )
    
    response = operations.result(timeout=360)

    text = []
    
    for i, result in enumerate(response.results):
            alternative = result.alternatives[0]
            alter=ConvertDate(alternative.transcript+ '\n')
#             print("-" * 20)
#             print("First alternative of result {}".format(i))
#             print("Transcript: {}".format(alternative.transcript))
            text.append(alter)
    return text

In [5]:
def Transcribe_Audios(Folder,config):
    audios = os.listdir(Folder)
    text = []
    i = 0
    while i <= len(audios)-1:
        audio = os.path.join(Folder,"{}.wav".format(i))
        length = len(AudioSegment.from_wav(audio))
        print('Processing : ', audio)
        if length < (60 * 1000):
            text.append(Transcribe_Short_Audio(audio,config))
        else:
            text.append(Transcribe_Long_Audio(audio,config))
        i += 1
    return text

In [6]:
def is_digit(word):
    try:
        int(word)
        return True
    except ValueError:
        pass
    return False

def ConvertDate(text):
    month=' tháng '
    year=' năm '
    for index in range(0,len(text)):
        try:
            if (text.index(month,index)==index):
                dateNum = text[index -1]
                monthNum = text[index + len(month)]
                if is_digit(dateNum) and is_digit(monthNum):
                    text=text[:index] + text[index+len(month)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
        try:
            if (text.index(year,index)==index):
                monthNum = text[index -1]
                yearNum = text[index + len(year)]
                if is_digit(monthNum) and is_digit(yearNum):
                    text=text[:index] + text[index+len(year)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
    return text

In [7]:
Folder = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Result2'
Result_txt = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\TrackRecord.txt'

rate, channel = frame_rate_channel(Folder + '\\0.wav')
config = Config_GGC(sample_rate_hertz = rate,
                audio_channel_count = channel)
text = Transcribe_Audios(Folder,config)

with open(Result_txt, 'w+', encoding='utf-8') as f:
    for t in text:
        f.writelines(t)
        f.write('\n')

C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\0.wav
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\0.wav
File C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\0.wav uploaded to Audio_wav.
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\1.wav
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\2.wav
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\3.wav
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\4.wav
Processing :  C:\Users\quang\Data Analysis\STECH\Speech-To-Text\Speech-Recognition\Multiple_speaker_detection\Result2\5.wav
Processi